In [1]:
import torchani
from ase.io import read, write
import torch
import sys
import ase
from ase.io import read, write
import matplotlib.pyplot as plt
from ase.units import fs
from copy import deepcopy
import torchani
import numpy as np

/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")


In [2]:
ani2x = torchani.models.ANI2x()
al = read("/Users/Cas/Work/HMD/PEG3/peg_all.xyz", ":")
atoms = al[1]

/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/


In [96]:
def get_comm_energy_forces(ani2x, at):
    Fs = []

    for i in range(8):
        calc = ani2x[i].ase()
        at.set_calculator(calc)
        Fs.append(at.get_forces())

    model = torchani.models.ANI2x(periodic_table_index=True)
    energy = model((species, coordinates)).energies

    derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
    force = -derivative

    _, atomic_energies = model.atomic_energies((species, coordinates), average=False)

    Es = atomic_energies.detach().numpy() * ase.units.Hartree

    #print(Es)
    meanE = np.mean(Es, axis=0)
    meanF = np.mean(Fs, axis=0)

    varE = np.var(Es, axis=0)
    varF = np.sum([2 * (Es[i] - meanE) * (Fs[i] - meanF).T for i in range(8)], axis=0)/8

    return varE, varF.T

In [97]:
varE, varF =get_comm_energy_forces(ani2x, al[1])

/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/


In [98]:
varE

array([[0.00730689, 0.0378946 , 0.03482063, 0.03023421, 0.03811524,
        0.03304666, 0.02649571, 0.03903259, 0.03831865, 0.03331003,
        0.03716946, 0.03967167, 0.00731955, 0.00379116, 0.00561245,
        0.00588754, 0.00671736, 0.00471105, 0.00565585, 0.00586362,
        0.00404982, 0.00586986, 0.00593299, 0.00611452, 0.00652284,
        0.00611278, 0.0056662 , 0.00594008, 0.00619087, 0.00604187,
        0.00254323]], dtype=float32)

In [88]:
def get_comm_energy_forces_old(ani2x, at):
    Es = []
    Fs = []

    for i in range(8):
        calc = ani2x[i].ase()
        at.set_calculator(calc)
        Es.append(at.get_potential_energy())
        Fs.append(at.get_forces())

    meanE = np.mean(Es)
    meanF = np.mean(Fs, axis=0)

    varE = np.sum([ (Es[i] - meanE)**2 for i in range(8)])/8
    varF = np.sum([2 * (Es[i] - meanE) * (Fs[i] - meanF) for i in range(8)], axis=0)/8

    return meanE, varE, meanF, varF

In [90]:
Es, meanE, meanF, varF = get_comm_energy_forces_old(ani2x, al[1])

In [91]:
varF

array([[-4.80451854e-04,  7.00486067e-04,  4.69138526e-04],
       [ 9.24542779e-04, -4.18823329e-04,  3.00642161e-04],
       [-2.43357557e-04, -1.09727858e-04, -1.02221046e-03],
       [ 6.05150126e-04,  1.28954800e-03, -1.33168604e-03],
       [ 1.86563848e-04, -1.50926702e-03, -3.67926492e-04],
       [-4.64132027e-04,  9.31467337e-04, -5.51920792e-04],
       [-8.16976608e-05, -1.06151961e-03, -1.38978008e-03],
       [ 1.33588415e-04, -1.14097726e-03,  4.90767998e-04],
       [-7.46307836e-04, -2.04633619e-03,  1.89210405e-05],
       [-4.22679354e-04,  6.14297343e-04,  1.07659504e-03],
       [ 7.69879553e-04, -2.17020279e-04,  3.25856017e-05],
       [ 3.52789531e-04, -3.69130459e-04,  1.20519726e-04],
       [-1.17017119e-03,  7.64853496e-04, -1.80693707e-04],
       [ 6.05734065e-04,  4.27329040e-04, -2.83531292e-04],
       [ 1.73022228e-04, -1.71149411e-04,  1.88183094e-05],
       [-7.03681377e-04,  1.85708574e-04,  2.40772832e-04],
       [ 2.68522213e-04, -2.82690569e-04

In [5]:
def get_comm_energy_forces(ani2x, at):
    Es = []
    Fs = []

    species = torch.tensor([at.get_atomic_numbers()], dtype=torch.long)
    coordinates = torch.tensor([at.get_positions()], requires_grad=True, dtype=torch.float32)

    model = torchani.models.ANI2x(periodic_table_index=True)
    energy = model((species, coordinates)).energies

    derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
    force = -derivative

    _, atomic_energies = model.atomic_energies((species, coordinates))

    for i in range(8):
        derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
        force = -derivative
        Fs.append(at.get_forces())

    print(Fs)

    # meanE = np.mean(Es)
    # meanF = np.mean(Fs, axis=0)

    # varE = np.sum([ (Es[i] - meanE)**2 for i in range(8)])/8
    # varF = np.sum([2 * (Es[i] - meanE) * (Fs[i] - meanF) for i in range(8)], axis=0)/8

In [6]:
get_comm_energy_forces(ani2x, al[1])

/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

In [8]:
at = al[100]

In [22]:
species = torch.tensor([at.get_atomic_numbers()], dtype=torch.long)
coordinates = torch.tensor([at.get_positions()], requires_grad=True, dtype=torch.float32)

Fs = []

for i in range(8):
    model = torchani.models.ANI2x(periodic_table_index=True)[i]
    energy = model((species, coordinates)).energies

    derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
    force = -derivative
    Fs.append(force)
#_, atomic_energies = model.atomic_energies((species, coordinates))

/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/
/Users/Cas/miniconda3/lib/python3.8/site-packages/torchani-2.3.dev60+ga85e330-py3.8.egg/torchani/resources/


In [23]:
Fs

[tensor([[[ 0.0744, -0.0050,  0.0048],
          [-0.1009,  0.0038,  0.0315],
          [ 0.0841, -0.0899, -0.0546],
          [ 0.0294,  0.1617, -0.0364],
          [ 0.0353, -0.1241,  0.0376],
          [-0.0835, -0.0377,  0.0721],
          [-0.0466,  0.0074, -0.0262],
          [ 0.0660, -0.0370,  0.0539],
          [ 0.0229,  0.0313, -0.0036],
          [ 0.0260, -0.0253,  0.0304],
          [ 0.0411,  0.0163, -0.0350],
          [-0.1299,  0.0152,  0.0287],
          [ 0.0651, -0.1408, -0.0737],
          [-0.0103, -0.0099, -0.0009],
          [-0.0060,  0.0092, -0.0064],
          [ 0.0093,  0.0165, -0.0208],
          [-0.0115,  0.0042,  0.0554],
          [-0.0431,  0.0271,  0.0208],
          [-0.0082,  0.0146, -0.0106],
          [-0.0373, -0.0219, -0.0085],
          [ 0.0565,  0.0883, -0.0539],
          [ 0.0033, -0.0065,  0.0203],
          [-0.0228, -0.0258,  0.0009],
          [ 0.0003,  0.0448, -0.0380],
          [-0.0070, -0.0355,  0.0014],
          [-0.0381, -0.00

In [83]:
coordinates = torch.tensor([at.get_positions()], requires_grad=True, dtype=torch.float32)

In [84]:
model = torchani.models.ANI2x(periodic_table_index=True)

energy = model((species, coordinates)).energies

In [85]:
energy

tensor([-691.5021], dtype=torch.float64, grad_fn=<AddBackward0>)

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###############################################################################
# Let's now load the built-in ANI-1ccx models. The builtin ANI-1ccx contains 8
# models trained with diffrent initialization. Predicting the energy and force
# using the average of the 8 models outperform using a single model, so it is
# always recommended to use an ensemble, unless the speed of computation is an
# issue in your application.
#
# The ``periodic_table_index`` arguments tells TorchANI to use element index
# in periodic table to index species. If not specified, you need to use
# 0, 1, 2, 3, ... to index species
model = torchani.models.ANI2x(periodic_table_index=True).to(device)

###############################################################################
# Now let's define the coordinate and species. If you just want to compute the
# energy and force for a single structure like in this example, you need to
# make the coordinate tensor has shape ``(1, Na, 3)`` and species has shape
# ``(1, Na)``, where ``Na`` is the number of atoms in the molecule, the
# preceding ``1`` in the shape is here to support batch processing like in
# training. If you have ``N`` different structures to compute, then make it
# ``N``.
#
# .. note:: The coordinates are in Angstrom, and the energies you get are in Hartree
coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True, device=device)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]], device=device)

###############################################################################
# Now let's compute energy and force:
energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative

###############################################################################
# And print to see the result:
print('Energy:', energy.item())
print('Force:', force.squeeze())

###############################################################################
# you can also get the atomic energies (WARNING: these have no physical
# meaning) by calling:
_, atomic_energies = model.atomic_energies((species, coordinates))


Energy: -40.459790705366636
Force: tensor([[ 0.0478, -0.1304, -0.0551],
        [-0.1353,  0.1581, -0.0776],
        [ 0.0804, -0.0388,  0.0387],
        [ 0.0254,  0.0076,  0.0433],
        [-0.0183,  0.0035,  0.0508]])


ModuleAttributeError: 'BuiltinEnsemble' object has no attribute 'atomic_energies'

In [ ]:
atomic_energies

In [120]:
species

tensor([[6, 1, 1, 1, 1]])

In [10]:
coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True, device=device)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]], device=device)

###############################################################################
# Now let's compute energy and force:
energy = model((species, coordinates)).energies

In [11]:
energy

tensor([-40.4598], dtype=torch.float64, grad_fn=<AddBackward0>)

In [12]:
calc = ani2x[1].ase()
at.set_calculator(calc)

NameError: name 'ani2x' is not defined

In [14]:
# Let's now manually specify the device we want TorchANI to run:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

###############################################################################
# Let's now load the built-in ANI-1ccx models. The builtin ANI-1ccx contains 8
# models trained with diffrent initialization. Predicting the energy and force
# using the average of the 8 models outperform using a single model, so it is
# always recommended to use an ensemble, unless the speed of computation is an
# issue in your application.
#
# The ``periodic_table_index`` arguments tells TorchANI to use element index
# in periodic table to index species. If not specified, you need to use
# 0, 1, 2, 3, ... to index species
model = torchani.models.ANI2x(periodic_table_index=True).to(device)

###############################################################################
# Now let's define the coordinate and species. If you just want to compute the
# energy and force for a single structure like in this example, you need to
# make the coordinate tensor has shape ``(1, Na, 3)`` and species has shape
# ``(1, Na)``, where ``Na`` is the number of atoms in the molecule, the
# preceding ``1`` in the shape is here to support batch processing like in
# training. If you have ``N`` different structures to compute, then make it
# ``N``.
#
# .. note:: The coordinates are in Angstrom, and the energies you get are in Hartree
coordinates = torch.tensor([[[0.03192167, 0.00638559, 0.01301679],
                             [-0.83140486, 0.39370209, -0.26395324],
                             [-0.66518241, -0.84461308, 0.20759389],
                             [0.45554739, 0.54289633, 0.81170881],
                             [0.66091919, -0.16799635, -0.91037834]]],
                           requires_grad=True, device=device)
# In periodic table, C = 6 and H = 1
species = torch.tensor([[6, 1, 1, 1, 1]], device=device)

###############################################################################
# Now let's compute energy and force:
energy = model((species, coordinates)).energies
derivative = torch.autograd.grad(energy.sum(), coordinates)[0]
force = -derivative

###############################################################################
# And print to see the result:
print('Energy:', energy.item())
print('Force:', force.squeeze())

###############################################################################
# you can also get the atomic energies (WARNING: these have no physical
# meaning) by calling:
_, atomic_energies = model.atomic_energies((species, coordinates))

Energy: -40.459790705366636
Force: tensor([[ 0.0478, -0.1304, -0.0551],
        [-0.1353,  0.1581, -0.0776],
        [ 0.0804, -0.0388,  0.0387],
        [ 0.0254,  0.0076,  0.0433],
        [-0.0183,  0.0035,  0.0508]])


ModuleAttributeError: 'BuiltinEnsemble' object has no attribute 'atomic_energies'

In [15]:
torchani.models.BuiltinEnsemble()

_, atomic_energies = model.atomic_energies((species, coordinates))

TypeError: __init__() missing 8 required positional arguments: 'species_converter', 'aev_computer', 'neural_networks', 'energy_shifter', 'species_to_tensor', 'consts', 'sae_dict', and 'periodic_table_index'

In [25]:
atomic_energies

tensor([[-1036.1987,   -15.8001,   -16.0767,   -16.4552,   -16.4363]],
       grad_fn=<MulBackward0>)

In [30]:
atoms.calculator(calc)

AttributeError: 'Atoms' object has no attribute 'calculator'

In [29]:
print(atoms.get_potential_energy())

-18816.786657402994


In [31]:
atomic_energies

tensor([[-1036.1987,   -15.8001,   -16.0767,   -16.4552,   -16.4363]],
       grad_fn=<MulBackward0>)

In [32]:
import numpy as np

In [35]:
np.sum(atomic_energies.detach().numpy())

-1100.9669

In [36]:
len(atoms)

31

In [38]:
_, atomic_energies = model.atomic_energies((species, coordinates), average=False)

In [39]:
atomic_energies 

tensor([[[-38.0841,  -0.5797,  -0.5898,  -0.6034,  -0.6027]],

        [[-38.0564,  -0.5861,  -0.5958,  -0.6112,  -0.6099]],

        [[-38.0744,  -0.5842,  -0.5930,  -0.6044,  -0.6037]],

        [[-38.0934,  -0.5769,  -0.5877,  -0.6012,  -0.6010]],

        [[-38.0904,  -0.5771,  -0.5880,  -0.6023,  -0.6016]],

        [[-38.0684,  -0.5833,  -0.5935,  -0.6079,  -0.6070]],

        [[-38.0826,  -0.5798,  -0.5901,  -0.6040,  -0.6033]],

        [[-38.0870,  -0.5781,  -0.5885,  -0.6033,  -0.6030]]],
       grad_fn=<AddBackward0>)

In [40]:
print('Atomic energies of first model, for species 6 1 1 1 1', atomic_energies[0, :, :])

Atomic energies of first model, for species 6 1 1 1 1 tensor([[-38.0841,  -0.5797,  -0.5898,  -0.6034,  -0.6027]],
       grad_fn=<SliceBackward>)
